# 🐫 CAMEL Agentic Search Cookbook

You can also check this cookbook in colab [here](https://colab.research.google.com/drive/1-qZEzCvKhuuLA8zkCci0OeL0utgKyNOs?usp=sharing)

<div class="align-center">
  <a href="https://www.camel-ai.org/"><img src="https://i.postimg.cc/KzQ5rfBC/button.png"width="150"></a>
  <a href="https://discord.camel-ai.org"><img src="https://i.postimg.cc/L4wPdG9N/join-2.png"  width="150"></a></a>
  
⭐ <i>Star us on [*Github*](https://github.com/camel-ai/camel), join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)
</div>

This notebook demonstrates how to set up and leverage CAMEL's agent to search the answer for an input question.

In this notebook, you'll explore:

*   **CAMEL**: A powerful multi-agent framework that enables Retrieval-Augmented Generation and multi-agent role-playing scenarios, allowing for sophisticated AI-driven tasks.
*   **`ChatAgent`**: The ChatAgent class.
*   **`RolePlaying`**: A unique cooperative agent framework of CAMEL, which enables multi-agent collaboration to solve problems through role play.
*   **`CriticAgent`**: A helper agent that helps to select proposals and provide feedbacks in a role playing session.
*   **`SearchToolkit`**: A class representing a toolkit for web search.


This setup not only demonstrates a practical application but also serves as a flexible framework that can be adapted for various scenarios requiring different toolkits.

## 📦 Installation

First, install the CAMEL package with all its dependencies:

In [ ]:
!pip install "camel-ai[all]" --no-deps
!pip install colorama tiktoken duckduckgo_search

## 🔑 Setting Up API Keys

You'll need to set up your API keys for OpenAI. This ensures that the tools can interact with external services securely.

In [ ]:
import os
from getpass import getpass
# Prompt for the API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your API key: ··········


If you need to use Google search tool, make sure to set up the Google API and Search engine

In [ ]:
google_api_key = getpass('Enter your Google API key: ')
os.environ["GOOGLE_API_KEY"] = google_api_key

search_engine_id = getpass('Enter your Google Search Engine ID: ')
os.environ["SEARCH_ENGINE_ID"] = search_engine_id

## 🌐 Use Base ChatAgent with SearchToolkit to Search Answer for a Query

*In this section, we'll demonstrate how to use existing `SearchToolkit` to find the answer to an input query repeatly until it finds the answer*

### Prepare your environment by initializing necessary imports from CAMEL

In [ ]:
from camel.agents import ChatAgent
from camel.toolkits import SearchToolkit, FunctionTool
from camel.types import ModelPlatformType, ModelType
from camel.configs import ChatGPTConfig
from camel.models import ModelFactory
from colorama import Fore
import asyncio

### CAMEL `ChatAgent` repeatly call search API to find the answer based on a given query. Critic agent will judge the answer and determine when to stop.

Create a `ChatAgent`

In [ ]:
# Initialize the model
model = ModelFactory.create(
    model_type=ModelType.GPT_4O_MINI,
    model_platform=ModelPlatformType.OPENAI,
    model_config_dict=ChatGPTConfig(temperature=0.0).as_dict(),
)

# Create chat agent
chat_system_message = (
    """You are a helpful assistant that can use a search
      engine to answer questions.
      Provide the best possible answer for each query."""
      )

search_agent = ChatAgent(
    system_message=chat_system_message,
    model=model,
    # SearchToolkit().search_google for Google search, and it requires api keys
    tools=[FunctionTool(SearchToolkit().search_duckduckgo)],
)



Create a critic agent based `ChatAgent`

In [ ]:
critic_system_message = (
    """
    You are a highly critical evaluator. Evaluate the following answer to
the question: '{query}'.
Answer: {content}

Consider if the answer directly and completely addresses the question.

"""
)
critic_agent = ChatAgent(system_message=critic_system_message, model=model)

The following function repeatly call search API to find the answer based on a given query.

In [ ]:
async def search_with_critic(
    agent: ChatAgent,
    critic_agent: ChatAgent,
    query: str,
    max_attempts: int = 3
) -> str:
  r"""Repeatly call search API to find the answer based on a given query.

  Args:
    agent (ChatAgent): The ChatAgent with the search tool.
    critic_agent (CriticAgent): The CriticAgent to critique the answers.
    initial_query (str): The initial search query.
    max_attempts (int): The maximum number of search attempts.

  Returns:
    The answer to the query.
  """
  for attempt in range(max_attempts):
    print(f"Attempt {attempt + 1}: Searching for '{query}'...")
    response = agent.step(query)
    content = response.msgs[0].content

    print("Agent's Response:", content)
    # Critique the agent's response

    critic_prompt = f"""
      You are a highly critical evaluator. Evaluate the following answer to
      the question: '{query}'.
      Answer: {content}

      Consider if the answer completely addresses the question.

      Respond with ONLY 'YES' if the answer meets the above criteria,
      otherwise respond with ONLY 'NO'.
      """
    critic_response = critic_agent.step(input_message=critic_prompt,
                                        response_format=None)
    critic_content = critic_response.msgs[0].content.strip().upper()
    print("Critic's Response:", critic_content)

    if "YES" in critic_content:
        print(Fore.RED + "Satisfactory answer found!")
        return content
    await asyncio.sleep(1)
  return f"Search incomplete after {max_attempts} attempts. \
  Last answer: {content}"



In [ ]:
# Define the initial query
query = "What is quantum mechanics, and what are the applications"

# Perform the search with the critic agent determining the final answer
final_answer = await search_with_critic(search_agent, critic_agent, query)

# Print the final answer
print("\nFinal Answer:", final_answer)

Attempt 1: Searching for 'What is quantum mechanics, and what are the applications'...
Agent's Response: ### What is Quantum Mechanics?

Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and light at the atomic and subatomic levels. It is essential for understanding the properties of molecules, atoms, and their constituents, such as electrons, protons, and neutrons. Quantum mechanics introduces concepts that are often counterintuitive, such as superposition (where particles can exist in multiple states at once), entanglement (where particles become interconnected in ways that the state of one can depend on the state of another, regardless of distance), and tunneling (where particles can pass through barriers that they classically shouldn't be able to).

For more detailed information, you can refer to the following resources:
- [Wikipedia on Quantum Mechanics](https://en.wikipedia.org/wiki/Quantum_mechanics)
- [Science News Today on Quantum Mecha

## 🤖🤖🤖 Use AI Society with SearchToolkit to Seary Answer for a Query

In [ ]:
from camel.societies import RolePlaying

In [ ]:
# set the query
query = "explain quantum mechanic and its applications"

In [ ]:
# role playing
role_play = RolePlaying(
    assistant_role_name="Searcher",
    user_role_name="User",
    critic_role_name="Critic",
    with_critic_in_the_loop=True,
    critic_criteria="""Say 'STOP' if the previousresponse is complete
                    for the task, 'CONTINUE' otherwise.""",
    model = model,
    assistant_agent_kwargs=dict(
        tools = [SearchToolkit().search_duckduckgo]
    ),
    user_agent_kwargs=dict(
        model = model,
    ),
    critic_kwargs=dict(
        model = model,
    ),
    task_prompt=query,
)

In [ ]:
# limit for the chat turn
chat_turn_limit = 5

In [ ]:
n = 0
input_msg = role_play.init_chat()
while n < chat_turn_limit:
  n +=1
  assistant_response, user_response = role_play.step(input_msg)
  if assistant_response.terminated:
    print("Assistant terminated the conversation")
    break
  if user_response.terminated:
    print("User terminated the conversation")
    break
  # Print the assistant's response
  print(f"Assistant: {assistant_response.msgs[0].content}")

  # Critic evaluates the response
  if role_play.critic is not None:
      critic_response = role_play.critic.step(assistant_response.msgs[0])
      critic_feedback = critic_response.msgs[0].content
      print(f"Critic Feedback: {critic_feedback}")


  if "STOP" in critic_response.msgs[0].content.upper():
    print(Fore.RED + "Satisfactory answer found!")
    print(assistant_response.msgs[0].content)
    break
  input_msg = assistant_response.msgs[0]

Assistant: Solution: 

The principle of superposition is a fundamental concept in quantum mechanics that states that a quantum system can exist in multiple states at the same time until it is measured. This principle is a departure from classical physics, where systems are typically found in one definite state.

In quantum mechanics, a particle, such as an electron, can be in a superposition of different states, represented mathematically by a wave function. For example, an electron can be in multiple positions or have different energy levels simultaneously. The wave function describes the probabilities of finding the particle in each of these states when a measurement is made.

A classic illustration of superposition is the thought experiment known as Schrödinger's cat. In this scenario, a cat is placed in a sealed box with a radioactive atom, a Geiger counter, and a vial of poison. If the atom decays, the Geiger counter triggers the release of the poison, killing the cat. Until the b

## 🌟 Highlights

This notebook has guided you through setting up and running agentic search using `SearchToolkit` and `ChatAgent`.

Key tools utilized in this notebook include:

*   **CAMEL**: A powerful multi-agent framework that enables Retrieval-Augmented Generation and multi-agent role-playing scenarios, allowing for sophisticated AI-driven tasks.
*   **`ChatAgent`**: The ChatAgent class.
*   **`RolePlaying`**: A unique cooperative agent framework of CAMEL, which enables multi-agent collaboration to solve problems through role play.
*   **`CriticAgent`**: A helper agent that helps to select proposals and provide feedbacks in a role playing session.
*   **`SearchToolkit`**: A class representing a toolkit for web search.

This comprehensive setup allows you to adapt and expand the example for various scenario using other search tools.

**CAMEL also provides other Toolkits, for more information please check [here](https://colab.research.google.com/drive/1f1jYwDy6pB8QB6c_UdoBvFx6wr6kA5xq?usp=sharing)**

That's everything: Got questions about 🐫 CAMEL-AI? Join us on [Discord](https://discord.camel-ai.org)! Whether you want to share feedback, explore the latest in multi-agent systems, get support, or connect with others on exciting projects, we’d love to have you in the community! 🤝

Check out some of our other work:

1. 🐫 Creating Your First CAMEL Agent [free Colab](https://docs.camel-ai.org/cookbooks/create_your_first_agent.html)

2.  Graph RAG Cookbook [free Colab](https://colab.research.google.com/drive/1uZKQSuu0qW6ukkuSv9TukLB9bVaS1H0U?usp=sharing)

3. 🧑‍⚖️ Create A Hackathon Judge Committee with Workforce [free Colab](https://colab.research.google.com/drive/18ajYUMfwDx3WyrjHow3EvUMpKQDcrLtr?usp=sharing)

4. 🔥 3 ways to ingest data from websites with Firecrawl & CAMEL [free Colab](https://colab.research.google.com/drive/1lOmM3VmgR1hLwDKdeLGFve_75RFW0R9I?usp=sharing)

5. 🦥 Agentic SFT Data Generation with CAMEL and Mistral Models, Fine-Tuned with Unsloth [free Colab](https://colab.research.google.com/drive/1lYgArBw7ARVPSpdwgKLYnp_NEXiNDOd-?usp=sharingg)

Thanks from everyone at 🐫 CAMEL-AI


<div class="align-center">
  <a href="https://www.camel-ai.org/"><img src="https://i.postimg.cc/KzQ5rfBC/button.png"width="150"></a>
  <a href="https://discord.camel-ai.org"><img src="https://i.postimg.cc/L4wPdG9N/join-2.png"  width="150"></a></a>
  
⭐ <i>Star us on [*Github*](https://github.com/camel-ai/camel), join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)
</div>
